In [1]:
import os, sys
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULES_PATH = os.path.join(BASE_PATH, "modules")
TF_PATH = os.path.join(BASE_PATH, "tf_al")

In [3]:
sys.path.append(MODULES_PATH)
sys.path.append(TF_PATH)

In [4]:
from tf_al import Config, Pool, Dataset
from tf_al.wrapper import McDropout
from models import fchollet_cnn, setup_growth, disable_tf_logs

In [5]:
SEED = 84123
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Prepare datasets

In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
datagen.fit(x_train)
x_train = datagen.standardize(x_train.astype(np.float32))

In [7]:
datagen.fit(x_test)
x_test = datagen.standardize(x_test.astype(np.float32))

y_train = y_train.flatten()
y_test = y_test.flatten()

init_pool_size = 20
dataset = Dataset(
    x_train, y_train,
    test=(x_test, y_test),
    init_size=init_pool_size
)

In [8]:
init_size = 1000
pool = Pool(x_train, y_train)
pool.init(init_size)
x_lab, y_lab = pool.get_labeled_data()

In [9]:
disable_tf_logs()
setup_growth()

1 Physical GPU's,  1 Logical GPU's


## Create Mc Dropout model

In [10]:
from tensorflow.keras.optimizers import Adam

verbose = False
optimizer = Adam()
loss = "sparse_categorical_crossentropy"
metrics = [keras.metrics.SparseCategoricalAccuracy()]
num_classes = len(list(np.unique(y_test)))
input_shape = tuple(x_train.shape[1:])

batch_size = 10
sample_size = 25
config = Config(
    fit={"epochs": 200, "batch_size": batch_size},
    query={"sample_size": sample_size},
    eval={"batch_size": batch_size, "sample_size": sample_size}
)

In [11]:
base_model = fchollet_cnn(input_shape=input_shape, output=num_classes)
mc_model = McDropout(base_model, config=config, verbose=verbose)
mc_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [12]:
mc_model.fit(x_lab, y_lab)

Epoch 1/200
100/100 [==============================] - 0s 3ms/step - loss: 2.2157 - sparse_categorical_accuracy: 0.1860
Epoch 2/200
100/100 [==============================] - 0s 3ms/step - loss: 1.9471 - sparse_categorical_accuracy: 0.2930
Epoch 3/200
100/100 [==============================] - 0s 3ms/step - loss: 1.6837 - sparse_categorical_accuracy: 0.4130
Epoch 4/200
100/100 [==============================] - 0s 3ms/step - loss: 1.3886 - sparse_categorical_accuracy: 0.5140
Epoch 5/200
100/100 [==============================] - 0s 2ms/step - loss: 1.0858 - sparse_categorical_accuracy: 0.6190
Epoch 6/200
100/100 [==============================] - 0s 3ms/step - loss: 0.9091 - sparse_categorical_accuracy: 0.6890
Epoch 7/200
100/100 [==============================] - 0s 3ms/step - loss: 0.7147 - sparse_categorical_accuracy: 0.7500
Epoch 8/200
100/100 [==============================] - 0s 3ms/step - loss: 0.5480 - sparse_categorical_accuracy: 0.7970
Epoch 9/200
100/100 [===================

100/100 [==============================] - 0s 3ms/step - loss: 0.0467 - sparse_categorical_accuracy: 0.9820
Epoch 69/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0595 - sparse_categorical_accuracy: 0.9820
Epoch 70/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0881 - sparse_categorical_accuracy: 0.9730
Epoch 71/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0656 - sparse_categorical_accuracy: 0.9770
Epoch 72/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0619 - sparse_categorical_accuracy: 0.9770
Epoch 73/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0729 - sparse_categorical_accuracy: 0.9750
Epoch 74/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0772 - sparse_categorical_accuracy: 0.9690
Epoch 75/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0649 - sparse_categorical_accuracy: 0.9780
Epoch 76/200
100/100 [=======================

100/100 [==============================] - 0s 3ms/step - loss: 0.0376 - sparse_categorical_accuracy: 0.9880
Epoch 136/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0464 - sparse_categorical_accuracy: 0.9870
Epoch 137/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0336 - sparse_categorical_accuracy: 0.9890
Epoch 138/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0519 - sparse_categorical_accuracy: 0.9870
Epoch 139/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0358 - sparse_categorical_accuracy: 0.9860
Epoch 140/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0387 - sparse_categorical_accuracy: 0.9910
Epoch 141/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0591 - sparse_categorical_accuracy: 0.9790
Epoch 142/200
100/100 [==============================] - 0s 3ms/step - loss: 0.0373 - sparse_categorical_accuracy: 0.9890
Epoch 143/200
100/100 [===============

In [13]:
mc_model.evaluate(x_test, y_test, batch_size=900)

ResourceExhaustedError: OOM when allocating tensor with shape[834,32,32,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

In [14]:
vgg_model = keras.applications.VGG16(
    weights=None,
    classes=10,
    input_shape=(32, 32, 3)
)
mc_model = McDropout(vgg_model, config=config, verbose=verbose)
mc_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [15]:
mc_model.fit(x_lab, y_lab)

Epoch 1/200
2/2 [==============================] - 0s 14ms/step - loss: 4.0522 - sparse_categorical_accuracy: 0.5250
Epoch 2/200
2/2 [==============================] - 0s 14ms/step - loss: 2.3154 - sparse_categorical_accuracy: 0.0500
Epoch 3/200
2/2 [==============================] - 0s 13ms/step - loss: 2.3096 - sparse_categorical_accuracy: 0.1000
Epoch 4/200
2/2 [==============================] - 0s 13ms/step - loss: 2.3099 - sparse_categorical_accuracy: 0.1000
Epoch 5/200
2/2 [==============================] - 0s 12ms/step - loss: 2.3164 - sparse_categorical_accuracy: 0.0000e+00
Epoch 6/200
2/2 [==============================] - 0s 12ms/step - loss: 2.3463 - sparse_categorical_accuracy: 0.1000
Epoch 7/200
2/2 [==============================] - 0s 12ms/step - loss: 2.3045 - sparse_categorical_accuracy: 0.1000
Epoch 8/200
2/2 [==============================] - 0s 12ms/step - loss: 2.3037 - sparse_categorical_accuracy: 0.1000
Epoch 9/200
2/2 [==============================] - 0s 11ms/s

Epoch 139/200
2/2 [==============================] - 0s 11ms/step - loss: 2.3032 - sparse_categorical_accuracy: 0.1000
Epoch 140/200
2/2 [==============================] - 0s 12ms/step - loss: 2.3029 - sparse_categorical_accuracy: 0.1000
Epoch 141/200
2/2 [==============================] - 0s 11ms/step - loss: 2.3030 - sparse_categorical_accuracy: 0.1000
Epoch 142/200
2/2 [==============================] - 0s 11ms/step - loss: 2.3031 - sparse_categorical_accuracy: 0.1000
Epoch 143/200
2/2 [==============================] - 0s 12ms/step - loss: 2.3032 - sparse_categorical_accuracy: 0.1000
Epoch 144/200
2/2 [==============================] - 0s 12ms/step - loss: 2.3031 - sparse_categorical_accuracy: 0.1000
Epoch 145/200
2/2 [==============================] - 0s 11ms/step - loss: 2.3038 - sparse_categorical_accuracy: 0.0500
Epoch 146/200
2/2 [==============================] - 0s 11ms/step - loss: 2.3030 - sparse_categorical_accuracy: 0.0500
Epoch 147/200
2/2 [=============================

In [16]:
mc_model.evaluate(x_test, y_test, sample_size=1, batch_size=200)

{'loss': 2.3026314, 'accuracy': 0.1}